In [1]:
import pandas as pd
import numpy as np
import json
import os

# custom functions
import functions

# bert and pytorch packages
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification

In [2]:
# load and unpack

fp = 'data/news/raw/netflix_general_news.json'

with open(fp) as json_file:
    data = json.load(json_file)
    news_df = pd.read_json(fp)
articles = functions.newsapiArticleUnpack(news_df)
articles

,author,content,description,publishedAt,source,title,url,urlToImage
0,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...
1,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...
2,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...
3,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...
4,newsfeedback@fool.com (Anders Bylund),Media-streaming veteran Netflix(NFLX -1.87%) i...,The digital video maven will report earnings n...,2022-07-12T14:05:22Z,"{'id': None, 'name': 'Motley Fool'}",Should You Buy Netflix Stock Before Next Week'...,https://www.fool.com/investing/2022/07/12/shou...,https://g.foolcdn.com/editorial/images/689628/...
...,...,...,...,...,...,...,...,...
121,Max Greve,"cyano66\r\nAs I write this, Netflix (NASDAQ:NF...",Netflix is a tech company because the key to i...,2022-08-01T16:53:44Z,"{'id': None, 'name': 'Seeking Alpha'}",Can Netflix Stock Recover After Subscriber Los...,https://seekingalpha.com/article/4528281-can-n...,https://static.seekingalpha.com/cdn/s3/uploads...
122,newsfeedback@fool.com (Anders Bylund),What happened\r\nShares of Netflix(NFLX -2.12%...,Here's why Netflix gained nearly 30% last mont...,2022-08-02T23:56:09Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Shares Rose 28.6% Last Month, But the ...",https://www.fool.com/investing/2022/08/02/netf...,https://g.foolcdn.com/editorial/images/693769/...
123,newsfeedback@fool.com (Tom Wilton),Streaming services saw a bump in sign-ups thro...,Peacock's moment might have arrived.,2022-08-02T11:29:47Z,"{'id': None, 'name': 'Motley Fool'}",This Streaming Stock Is Poised for Monster Gro...,https://www.fool.com/investing/2022/08/02/this...,https://g.foolcdn.com/editorial/images/693276/...
124,None,"NEW YORK, Aug. 4, 2022 /PRNewswire/ -- Investo...","NEW YORK, Aug. 4, 2022 /PRNewswire/ -- Investo...",2022-08-04T14:50:00Z,"{'id': None, 'name': 'PRNewswire'}",Thinking about trading options or stock in NVI...,https://www.prnewswire.com/news-releases/think...,https://mma.prnewswire.com/media/1333368/Inves...


In [3]:
# remove duplicates
articles.drop_duplicates(subset=['description'], inplace=True)

In [4]:
articles

,author,content,description,publishedAt,source,title,url,urlToImage
0,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...
1,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...
4,newsfeedback@fool.com (Anders Bylund),Media-streaming veteran Netflix(NFLX -1.87%) i...,The digital video maven will report earnings n...,2022-07-12T14:05:22Z,"{'id': None, 'name': 'Motley Fool'}",Should You Buy Netflix Stock Before Next Week'...,https://www.fool.com/investing/2022/07/12/shou...,https://g.foolcdn.com/editorial/images/689628/...
5,The Value Pendulum,stockcam\r\nElevator Pitch\r\n My investment r...,Netflix has been hit by increased subscriber c...,2022-07-12T14:09:47Z,"{'id': None, 'name': 'Seeking Alpha'}",Is Netflix Stock A Sell Before Upcoming Earnin...,https://seekingalpha.com/article/4522892-is-ne...,https://static.seekingalpha.com/cdn/s3/uploads...
8,None,"NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...","NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...",2022-07-13T14:55:00Z,"{'id': None, 'name': 'PRNewswire'}",Thinking about trading options or stock in Adv...,https://www.prnewswire.com/news-releases/think...,https://mma.prnewswire.com/media/1333368/Inves...
...,...,...,...,...,...,...,...,...
116,newsfeedback@fool.com (Tom Wilton),Walt Disney(DIS 1.14%) has a well-told history...,Walt Disney has a lot to teach Netflix about f...,2022-07-31T10:48:00Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Stock Is Playing Catch-Up to Disney, b...",https://www.fool.com/investing/2022/07/31/netf...,https://g.foolcdn.com/editorial/images/692688/...
118,Max Greve,"cyano66\r\nAs I write this, Netflix (NASDAQ:NF...",Netflix is a tech company because the key to i...,2022-08-01T16:53:44Z,"{'id': None, 'name': 'Seeking Alpha'}",Can Netflix Stock Recover After Subscriber Los...,https://seekingalpha.com/article/4528281-can-n...,https://static.seekingalpha.com/cdn/s3/uploads...
119,newsfeedback@fool.com (Anders Bylund),What happened\r\nShares of Netflix(NFLX -2.12%...,Here's why Netflix gained nearly 30% last mont...,2022-08-02T23:56:09Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Shares Rose 28.6% Last Month, But the ...",https://www.fool.com/investing/2022/08/02/netf...,https://g.foolcdn.com/editorial/images/693769/...
120,newsfeedback@fool.com (Tom Wilton),Streaming services saw a bump in sign-ups thro...,Peacock's moment might have arrived.,2022-08-02T11:29:47Z,"{'id': None, 'name': 'Motley Fool'}",This Streaming Stock Is Poised for Monster Gro...,https://www.fool.com/investing/2022/08/02/this...,https://g.foolcdn.com/editorial/images/693276/...


In [5]:
model_fp = 'models/classifier_model/finbert-sentiment'
text_fp = 'data/news/to_classify/content_processed.txt'

finbert = AutoModelForSequenceClassification.from_pretrained(model_fp,num_labels=3,cache_dir=None)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [6]:
# prep data for classification
text_dict = {}

for column in articles[['title','description']]:
    text_dict[column] = list(articles[column])

In [31]:
# classify data using pytorch get label probabilities

cat_dict = {}
neutral_dict = {}
positive_dict = {}
negative_dict = {}

labels = {0:'neutral', 1:'positive',2:'negative'}

for column in text_dict.keys():
    cat_list = []
    neutral = []
    positive = []
    negative = []
    for sentence in text_dict[column]:
        # send and receive data
        inputs = tokenizer(sentence, return_tensors="pt", padding=True)
        outputs = finbert(**inputs)[0]
        # only take the max prob to categorize
        sentiment = labels[np.argmax(outputs.detach().numpy())]
        print(sentence, '#####', sentiment)
        # store values
        cat_list.append(sentiment)
        neutral.append(outputs.detach().numpy()[0][0])
        positive.append(outputs.detach().numpy()[0][1])
        negative.append(outputs.detach().numpy()[0][2])

    cat_dict[column] = cat_list
    neutral_dict[column] = neutral
    positive_dict[column] = positive
    negative_dict[column] = negative

Why Is Everyone Talking About Netflix Stock? ##### negative
Why Netflix Stock Fell 4% Today ##### positive
Should You Buy Netflix Stock Before Next Week's Earnings Report? ##### negative
Is Netflix Stock A Sell Before Upcoming Earnings? Consider Short-Term Vs. Long-Term Outlook ##### negative
Thinking about trading options or stock in Advanced Micro Devices, Exxon Mobil, General Mills, Amazon, or Netflix? ##### negative
Is Netflix Stock Undervalued? ##### positive
Is Netflix Stock a Buy Before Earnings? ##### negative
Here's Why Netflix Stock Could Soar Next Week ##### negative
Better Streaming Stock: Disney vs Netflix ##### neutral
This Key Metric Could Make or Break Netflix Stock ##### negative
Better FAANG Stock Right Now: Alphabet or Netflix? ##### negative
Is Netflix Stock a Buy After Its Deal With Microsoft? ##### negative
Netflix Stock Has a Lot to Prove This Week ##### negative
Tesla, Netflix earnings, GameStop stock split, existing home sales top week ahead - Fox Business ####

In [33]:
# store values in df
for column in cat_dict.keys():
    articles[str(column)+"_sentiment_cat"] = cat_dict[column]
for column in neutral_dict.keys():
    articles[str(column)+"_sentiment_neutral"] = neutral_dict[column]
for column in positive_dict.keys():
    articles[str(column)+"_sentiment_positive"] = positive_dict[column]
for column in negative_dict.keys():
    articles[str(column)+"_sentiment_negative"] = negative_dict[column]
articles

,author,content,description,publishedAt,source,title,url,urlToImage,title_sentiment_cat,description_sentiment_cat,title_sentiment_pred,description_sentiment_pred,title_sentiment_neutral,description_sentiment_neutral,title_sentiment_positive,description_sentiment_positive,title_sentiment_negative,description_sentiment_negative
0,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...,negative,negative,1.082948,1.902008,-1.391915,0.311413,-0.004960,-2.104541,1.082948,1.902008
1,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...,positive,positive,-1.179389,-0.676670,-1.547374,-1.874073,2.198045,1.932919,-1.179389,-0.676670
4,newsfeedback@fool.com (Anders Bylund),Media-streaming veteran Netflix(NFLX -1.87%) i...,The digital video maven will report earnings n...,2022-07-12T14:05:22Z,"{'id': None, 'name': 'Motley Fool'}",Should You Buy Netflix Stock Before Next Week'...,https://www.fool.com/investing/2022/07/12/shou...,https://g.foolcdn.com/editorial/images/689628/...,negative,negative,2.427392,2.570103,-0.500017,-1.011215,-2.114805,-1.244079,2.427392,2.570103
5,The Value Pendulum,stockcam\r\nElevator Pitch\r\n My investment r...,Netflix has been hit by increased subscriber c...,2022-07-12T14:09:47Z,"{'id': None, 'name': 'Seeking Alpha'}",Is Netflix Stock A Sell Before Upcoming Earnin...,https://seekingalpha.com/article/4522892-is-ne...,https://static.seekingalpha.com/cdn/s3/uploads...,negative,positive,1.604493,-1.104882,-0.821234,-1.924304,-1.093194,2.227238,1.604493,-1.104882
8,None,"NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...","NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...",2022-07-13T14:55:00Z,"{'id': None, 'name': 'PRNewswire'}",Thinking about trading options or stock in Adv...,https://www.prnewswire.com/news-releases/think...,https://mma.prnewswire.com/media/1333368/Inves...,negative,negative,2.584871,2.411700,-1.115195,-0.729644,-1.108096,-1.705350,2.584871,2.411700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,newsfeedback@fool.com (Tom Wilton),Walt Disney(DIS 1.14%) has a well-told history...,Walt Disney has a lot to teach Netflix about f...,2022-07-31T10:48:00Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Stock Is Playing Catch-Up to Disney, b...",https://www.fool.com/investing/2022/07/31/netf...,https://g.foolcdn.com/editorial/images/692688/...,negative,negative,0.861580,2.377375,-1.297861,-0.006491,-0.078408,-2.598116,0.861580,2.377375
118,Max Greve,"cyano66\r\nAs I write this, Netflix (NASDAQ:NF...",Netflix is a tech company because the key to i...,2022-08-01T16:53:44Z,"{'id': None, 'name': 'Seeking Alpha'}",Can Netflix Stock Recover After Subscriber Los...,https://seekingalpha.com/article/4528281-can-n...,https://static.seekingalpha.com/cdn/s3/uploads...,neutral,negative,0.764911,2.184253,0.954058,-0.088306,-2.261653,-2.455376,0.764911,2.184253
119,newsfeedback@fool.com (Anders Bylund),What happened\r\nShares of Netflix(NFLX -2.12%...,Here's why Netflix gained nearly 30% last mont...,2022-08-02T23:56:09Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Shares Rose 28.6% Last Month, But the ...",https://www.fool.com/investing/2022/08/02/netf...,https://g.foolcdn.com/editorial/images/693769/...,neutral,neutral,-0.447852,0.357863,1.470942,1.353647,-1.281198,-2.124094,-0.447852,0.357863
120,newsfeedback@fool.com (Tom Wilton),Streaming services saw a bump in sign-ups thro...,Peacock's moment might have arrived

In [54]:
# Let's only consider the useful features of the data
out_articles = articles[['publishedAt','description_sentiment_cat', 'description_sentiment_neutral', 'description_sentiment_positive', 'description_sentiment_negative']]

# convert to datetime
out_articles['publishedAt'] = pd.to_datetime(out_articles['publishedAt'])

# group by and take the mean
daily_news = out_articles.groupby([out_articles['publishedAt'].dt.date]).mean()

# turn into categorical
sentiment_list = []

for row in daily_news.itertuples():
    predictions = row[1:]
    sentiment = labels[np.argmax(predictions)]
    sentiment_list.append(sentiment)

daily_news['daily_sentiment'] = sentiment_list

C:\Users\lapen\AppData\Local\Temp/ipykernel_21088/2948783557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_articles['publishedAt'] = pd.to_datetime(out_articles['publishedAt'])


In [56]:
# export non-aggregrated
out_articles.to_csv('data/news/classified/news_sentiment.csv')

In [55]:
# export daily
daily_news.to_csv('data/news/classified/news_mean_sentiment.csv')